In [1]:
import ssl
import urllib.request as web
import newspaper as np           # /!\ care with numpy alias
import feedparser as fp
import pandas as pd

import os
import json

In [4]:
! curl -O https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lemonde.json 
! curl -O https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.3M  100 14.3M    0     0   926k      0  0:00:15  0:00:15 --:--:--  965k0:00:08  0:00:08  986k5  0:00:13  0:00:02 1033k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   916  100   916    0     0   2726      0 --:--:-- --:--:-- --:--:--  2726


In [2]:
os.getcwd()

'/Users/abdoulayediallo/Documents/ENSAI/Cours/NLP/projets/web-scraping-lemonde'

In [6]:
ssl._create_default_https_context = ssl._create_unverified_context

# (b) recuperer le fichier RSS
url = "https://investir.lesechos.fr/RSS"
fn = "info-marches-investir-bourse-les-echos.xml"
data = fp.parse("/".join([url,fn]))

# (c) visualiser les elements du fichier RSS
print(data.feed.title)
print(" ")
print(data.feed.published)
print(" ")
# (d) iterer sur les entrees du flux RSS
for item in data.entries:
    print(item.title, item.published, item.link)
    break

Marchés, Indices, matières premières, analyses de séance, Taux
 
Wed, 03 Feb 2021 17:54:37 +0200
 
Paris Clôture : Le Cac 40 a été éclipsé par Mario Draghi, chargé de former un gouvernement en Italie Wed, 03 Feb 2021T17:40:00 +0200 https://investir.lesechos.fr/marches/actualites/le-cac-40-a-ete-eclipse-par-draghi-charge-de-former-un-gouvernement-en-italie-1947597.php?xtor=RSS-24


# TODO:
## 1 Data load and basic exploration
    - Lecture de la base de données existantes lemonde.json
    – regardez au passage la structure de ce fichier json et les informations qu’il contient

In [85]:
with open("data_out/lemonde.json", 'r') as l:
    lemonde = json.load(l)

with open("data_out/sources.json", 'r') as s:
    source = json.load(s)

print('Loaded', len(lemonde), "entries of lemonde")
print('Loaded', len(source), "entries of source")

Loaded 3041 entries of lemonde
Loaded 14 entries of source


### Head

In [5]:
[(k, v) for k, v in enumerate(lemonde.items())][0]

(0,
 ('https://www.lemonde.fr/international/article/2020/12/06/l-ue-met-en-place-un-nouveau-regime-de-sanctions-transversales_6062390_3210.html',
  {'title': 'L’UE met en place un nouveau régime de sanctions transversales',
   'date': '2020-12-06T14:00:08',
   'author': ['Laurence Girard',
    'Taha Oudghiri',
    'Vice-Président De L Association Marocaine Des Économistes D Entreprises Ameen',
    'Stéphane Foucart',
    'Jean-Pierre Stroobants',
    'Bruxelles',
    'Bureau Européen'],
   'category': 'Europe',
   'content': 'L’opposant russe Alexeï Navalny, le 22 août 2019 à Moscou. Alexander Zemlianichenko / AP\n\nLe « nouveau régime de sanctions transversales » que les ministres des affaires étrangères de l’Union européenne devraient approuver lundi 7 décembre, à Bruxelles, va peut-être clore des années de discussion et ramener l’Union à son exact niveau d’ambition. Qui ne peut plus être vraiment celui du « pouvoir transformateur » dont rêvaient certains, mais celui d’un acteur de r

### Identifier les informations disponibles dans le flux RSS de l’exemple ci-dessus et les  ́el ́ements correspondants dans la structure de donn ́ees retourn ́ees par fp.parse

In [86]:
kys = []
df = {}
for keys, values in lemonde.items():
    for k, v in values.items():
        kys.append(k)
        df[k] = v
        
# Sanity check
len(kys) == len(lemonde.items())*len(set(kys))

True

In [87]:
# Les attributs disponible dans la base
set(kys)

{'author', 'category', 'content', 'date', 'image_file', 'image_link', 'title'}

# pour chacun des flux RSS listés dans le fichier sources.json :

### – Lire les données du flux RSS
### – Scanner les articles et repérer ceux qui ne sont pas deja présent dans la base de données

# # TODO: 

## 2
    - Pour l’un des articles de votre flux RSS, regardez le code HTML de la page web. Pouvez-vous identifier les zones contenant le titre de l’article ? le texte de l’article ? En quoi le recours à la librairie newspaper3k est-il utile ? Quelles informations peut-on ainsi récupérer concernant un article ?TODO: 


**Réponse:**  
    - Le title est. dans `class="article__heading"`  
    - le text est dans `class="article__wrapper "`

- On peut récupere le caption des images dans `class="article__legend"`et le temps de lecture ncessaire dans `class="meta__reading-time meta__reading-time--header"`


## 3
Reprendre votre programme de scrapping et le compléter de maniere à mettre a jour votre base de données avec le texte des articles du flux RSS et le lien vers l’image illustrant l’article lorsqu’il y en a une. Vous pouvez inclure d’autres informations qui vous semblent utile pour un traitement ultérieur.

In [124]:
lemonde['https://www.lemonde.fr/international/article/2020/12/06/l-ue-met-en-place-un-nouveau-regime-de-sanctions-transversales_6062390_3210.html']["title"]

'L’UE met en place un nouveau régime de sanctions transversales'

In [80]:
# new df
for category, urls in source.items() :
    new_data = fp.parse(urls)
    print(new_data.entries[0].values())
    print("----------"*15)

dict_values(['Nathalie Becquart, première femme à avoir le droit de vote au synode des évêques', {'type': 'text/plain', 'language': None, 'base': 'https://www.lemonde.fr/europe/rss_full.xml', 'value': 'Nathalie Becquart, première femme à avoir le droit de vote au synode des évêques'}, 'Sat, 06 Feb 2021 14:56:57 +0100', time.struct_time(tm_year=2021, tm_mon=2, tm_mday=6, tm_hour=13, tm_min=56, tm_sec=57, tm_wday=5, tm_yday=37, tm_isdst=0), 'La Française a été nommée sous-secrétaire du synode, l’assemblée des évêques chargée d’étudier les grandes questions doctrinales au Vatican.', {'type': 'text/html', 'language': None, 'base': 'https://www.lemonde.fr/europe/rss_full.xml', 'value': 'La Française a été nommée sous-secrétaire du synode, l’assemblée des évêques chargée d’étudier les grandes questions doctrinales au Vatican.'}, 'https://www.lemonde.fr/societe/article/2021/02/06/nathalie-becquart-premiere-femme-a-avoir-le-droit-de-vote-au-synode-des-eveques_6069030_3224.html', True, 'https:/

In [115]:
linkto_update = []
for category, url in source.items():
    sources = fp.parse(url)
    for items in sources.entries:
        linkto_update.append(items.get("id"))
        
lien_a_ajouter = set(linkto_update) # liens a parser pour update la base lemonde
len(lien_a_ajouter)

216

In [125]:
notcommonlink = []
for lien in lien_a_ajouter: 
    if lien not in liens_existant: 
        notcommonlink.append(lien)

In [82]:
liens_existant = lemonde.keys() # Lien issue et existant dans notre base disponible, lemonde.

notcommonlink = []
for lien in lien_a_ajouter: 
    if lien not in liens_existant: 
        notcommonlink.append(lien)
    
# new df
for category, url in source.items() :
    new_data = fp.parse(url)
    
    for item in new_data.entries:
        #Boucle permettant le scan de l'article
        link = item.link
        index = link.find('.html')
        numero = link[index-7:index-1]
        
        if numero not in ids:
            #On verifie si l'article est deja presentou non dans la base
            article = np.Article(link)
            article.download()
            article.parse()
            #Afin d'avoir une base la plus complète pour la suite, nous avons décidé de garder l'ensemble de ces infos
            updatmonde[link] = {'title' : article.title,
                             'date' : article.publish_date.isoformat(), 
                             'author' : article.authors, 
                             'category' : category, 
                             'content' : article.text, 
                             'image_link' : article.top_image}
         
            

In [84]:
print(len(lemonde))

3041


In [13]:
# init data storage
print("Before update lenght df lemonde was {0}".format(len(lemonde)))

for link in links:
    if link not in list(lemonde.keys()):
        article = np.Article(link)
        article.download()
        article.parse()
        print(article)

        lemonde[link]["date"] = article.publish_date
        lemonde[link]["author"]= article.authors
        lemonde[link]["title"] = article.title
        # lemonde["link"]["category"] = article.c
        lemonde[link]["content"] = article.text
        lemonde[link]["image_link"] = article.top_image


print("After update lenght df lemonde is {0}".format(len(lemonde)))

Before update lenght df lemonde was 3041


KeyError: 'https://www.lemonde.fr/international/article/2021/02/03/en-italie-sergio-mattarella-recourt-a-mario-draghi-pour-trouver-une-issue-a-la-crise-politique_6068625_3210.html'

In [244]:
for link in links:
    if link not in list(lemonde.keys()):
        lemonde[link]["date"]

KeyError: 'https://www.lemonde.fr/international/article/2021/02/03/en-australie-au-moins-71-habitations-detruites-pres-de-perth-dans-un-incendie_6068573_3210.html'

In [205]:
# Sanity checker
data.shape[0] == len(links)

True

In [219]:
def scraper(sources, df, **dico):
    """ 
    Input:
    ------
    Source: json file containing links to parse
    dico: key value pairs of collected data. 
    
    Output:
    -------
    df: pandas dataframe where data will be stored
    """
    links = []
    for category, url in sources.items():
        sources = fp.parse(url)
        for items in sources.entries:
            links.append(items.get("id"))
    links = set(links) # remove from here dup links
    
    for link in links:
        article = np.Article(link)
        article.download()
        article.parse()
        dico["liens"] = link
        dico["title"] = article.title
        dico["text"] = article.text
        dico["image"] = article.top_image
        temp = pd.DataFrame(dico, index=range(0,len(links)))
        df = df.append(temp)
    df.drop_duplicates(subset=['liens'], inplace=True) 
    df.reset_index(inplace=True)
    
    return df

In [220]:
df = pd.DataFrame()
tmp = scraper(source, df)
tmp


,index,liens,title,text,image
0,0,https://www.lemonde.fr/international/article/2...,"En Australie, au moins 71 habitations détruite...",Les pompiers affrontent un incendie à Wooroloo...,https://img.lemde.fr/2021/02/02/249/0/3000/150...
1,0,https://www.lemonde.fr/politique/article/2021/...,Covid-19 : Emmanuel Macron veut rassurer à pro...,"Emmanuel Macron au palais de l’Elysée à Paris,...",https://img.lemde.fr/2021/02/02/0/0/8191/4091/...
2,0,https://www.lemonde.fr/economie/article/2021/0...,Chacun peut désormais noter son agence bancair...,A Paris. LOIC VENANCE / AFP\n\nLa folie de la ...,https://img.lemde.fr/2021/02/03/364/0/4368/218...
3,0,https://www.lemonde.fr/idees/article/2021/02/0...,"« Navalny, Blinken, Merkel et le gaz russe »","Chronique. En 1987, un jeune et brillant diplô...",https://img.lemde.fr/2021/01/26/308/0/3500/175...
4,0,https://www.lemonde.fr/economie/article/2021/0...,Fin de la précarité pour les femmes de chambre...,Une femme de chambre prépare une chambre d’hôt...,https://img.lemde.fr/2021/02/03/165/0/6016/300...
...,...,...,...,...,...
234,0,https://www.lemonde.fr/idees/article/2021/02/0...,« La France est aujourd’hui un pays de seconde...,Tribune. Tous les économistes reconnaissent qu...,https://img.lemde.fr/2021/01/28/911/0/5625/280...
235,0,https://www.lemonde.fr/sport/article/2021/02/0...,Football : huit personnes placées en détention...,Les services de police chiffrent à trois cents...,https://img.lemde.fr/2021/01/31/391/0/4692/234...
236,0,https://www.lemonde.fr/politique/article/2021/...,"Face aux polémiques, Gérald Darmanin tente d’a...",Gérald Darmanin lors d’une intervention de Jea...,https://img.lemde.fr/2021/01/27/571/0/6848/342...
237,0,https://www.lemonde.fr/planete/article/2021/01...,Covid-19 dans le monde : AstraZeneca devrait l...,Manifestation contre les mesures sanitaires à ...,https://img.lemde.fr/2021/01/22/75/0/3236/1615...
